In [5]:
%matplotlib notebook
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models
from tensorflow.keras.models import save_model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout

In [4]:
 pip install torchvision

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 660.6 kB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.2 MB 1.3 MB/s eta 0:00:01
   ----------- ---------------------------- 0.3/1.2 MB 2.5 MB/s eta 0:00:01
   ---------------------------- ----------- 0.8/1.2 MB 4.8 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 5.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
# PointCloud_Data 전처리 함수
def preprocess_point_cloud(file_path, length):
    with open(file_path, 'r') as file:
        data_point_cloud = [json.loads(line) for line in file]

    # Extract 'C' information from the JSON file
    processed_data = []

    for i in range(len(data_point_cloud)-1):
        if len(data_point_cloud[i+1]['C']) > length:
            data_point_cloud[i+1]['C'] = data_point_cloud[i+1]['C'][:length]
        elif len(data_point_cloud[i+1]['C']) < length:
            # Fill the missing part with 0
            data_point_cloud[i+1]['C'] += [0] * (length - len(data_point_cloud[i+1]['C']))

        processed_data.append(data_point_cloud[i+1]['C'])

    return processed_data


# skeleton_Data 전처리 함수
def preprocess_skeleton_data(file_path):
    with open(file_path, 'r') as file:
        skeleton_data = json.load(file)

    # Process skeleton data
    result_list = []
    for item in skeleton_data:
        x_value = float(item['X']) * 0.001
        y_value = float(item['Y']) * 0.001
        z_value = float(item['Z']) * 0.001

        result_list.append({'X': x_value, 'Y': y_value, 'Z': z_value})

    i = 0
    skeleton_data_list = []

    for item in result_list:
        k = [item['X'], item['Y'], item['Z']]
        skeleton_data_list.append(k)
        i += 1

    # Divide the data into groups of 22 joints per time step
    label_data = [skeleton_data_list[i:i+33] for i in range(0, len(skeleton_data_list), 33)]
    
    return label_data

def downsample_data(pointcloud_data, skeleton_data):
    if len(pointcloud_data) > len(skeleton_data):
        downsample_rate = len(pointcloud_data) // len(skeleton_data)
        downsampled_pointcloud_data = pointcloud_data[:len(skeleton_data) * downsample_rate:downsample_rate]
        processed_pointcloud_data = downsampled_pointcloud_data
        processed_skeleton_data = skeleton_data
    else:
        downsample_rate = len(skeleton_data) // len(pointcloud_data)
        downsampled_skeleton_data = skeleton_data[:len(pointcloud_data) * downsample_rate:downsample_rate]
        processed_pointcloud_data = pointcloud_data
        processed_skeleton_data = downsampled_skeleton_data

    return processed_pointcloud_data, processed_skeleton_data


def preprocess_and_split_data(point_cloud_path, skeleton_path, length):
    
    # PointCloud_Data_Walk 전처리
    processed_pointcloud_data = preprocess_point_cloud(point_cloud_path, length)

    # Skeleton_Data_Wlak 전처리
    processed_skeleton_data = preprocess_skeleton_data(skeleton_path)

    print(len(processed_pointcloud_data), len(processed_skeleton_data))

    # DataDownSampling
    processed_pointcloud_data, processed_skeleton_data = downsample_data(processed_pointcloud_data, processed_skeleton_data)

    print(len(processed_pointcloud_data), len(processed_skeleton_data))

    # 데이터 정규화
    scaler = StandardScaler()
    processed_pointcloud_data = [scaler.fit_transform(np.array(frame).reshape(-1, 3)).reshape(-1, 3) for frame in processed_pointcloud_data]

    return processed_pointcloud_data, processed_skeleton_data
    
# 포인트 클라우드 데이터 로드
file_path_point_cloud_Tpose = 'DataSet/OPenCVData/OpenCVPointCloud/PointcloudTpose.json'
file_path_point_cloud_Walk = 'DataSet/OPenCVData/OpenCVPointCloud/PointcloudWalk.json'

# skeleton Data Load
file_path_skeleton_Tpose = 'DataSet/OPenCVData/OpenCVSkeleton/SkeletonTpose.json'
file_path_skeleton_Walk = 'DataSet/OPenCVData/OpenCVSkeleton/SkeletonWalk.json'


# 원하는 길이로 변경
Length = 1500
    

# 전처리
processed_pointcloud_data_Tpose, processed_skeleton_data_Tpose = preprocess_and_split_data(file_path_point_cloud_Tpose, file_path_skeleton_Tpose, Length)
processed_pointcloud_data_Walk, processed_skeleton_data_Walk = preprocess_and_split_data(file_path_point_cloud_Walk, file_path_skeleton_Walk, Length)


# 데이터 정규화
#scaler = StandardScaler()
#processed_pointcloud_data_Walk = [scaler.fit_transform(np.array(frame).reshape(-1, 3)).reshape(-1, 3) for frame in processed_pointcloud_data_Walk]


#데이터 병합
Merged_processed_pointcloud_data = processed_pointcloud_data_Tpose + processed_pointcloud_data_Walk
Merged_processed_skeleton_data = processed_skeleton_data_Tpose + processed_skeleton_data_Walk

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(Merged_processed_pointcloud_data, Merged_processed_skeleton_data, test_size=0.2, random_state=42)




# CNN 모델 정의
input_layer = layers.Input(shape=(len(Merged_processed_pointcloud_data[0]), 3))
conv1d_layer = layers.Conv1D(64, 3, activation='relu', padding='same')(input_layer)
global_maxpool_layer = layers.GlobalMaxPooling1D()(conv1d_layer)
dense_layer = layers.Dense(99, activation='linear')(global_maxpool_layer)
dropout_layer = layers.Dropout(0.2)(dense_layer)
reshape_layer = layers.Reshape((33, 3))(dropout_layer)

# Additional layers
conv1d_layer_2 = layers.Conv1D(128, 3, activation='relu', padding='same')(reshape_layer)
global_maxpool_layer_2 = layers.GlobalMaxPooling1D()(conv1d_layer_2)
dense_layer_2 = layers.Dense(99, activation='linear')(global_maxpool_layer_2)
dropout_layer_2 = layers.Dropout(0.2)(dense_layer_2)
reshape_layer_2 = layers.Reshape((33, 3))(dropout_layer_2)

# Output layer
output_layer = layers.Dense(3, activation='linear')(reshape_layer_2)
# 모델 정의
model = models.Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 훈련
model.fit(np.array(X_train), np.array(y_train), epochs=500, batch_size=64, validation_data=(np.array(X_test), np.array(y_test)))

# 모델 저장
model.save('CNN_Model_Skeleton_OpenCV_Vf.h5')

# 모델 로드
loaded_model = load_model('CNN_Model_Skeleton_OpenCV_Vf.h5')

# 모델 평가를 수행
loaded_model.evaluate(np.array(X_test), np.array(y_test))

# 모델 예측을 수행
loaded_predictions = loaded_model.predict(np.array(X_test))

# 모델 평가
loss = model.evaluate(np.array(X_test), np.array(y_test))
print(f"Test Loss: {loss}")

predictions = model.predict(np.array(X_test))

print(len(predictions))


FileNotFoundError: [Errno 2] No such file or directory: 'DataSet/OPenCVData/OpenCVPointCloud/PointcloudTpose.json'